In [92]:
# %pip install google

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 2.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import googlemaps
from datetime import datetime

In [2]:
import pandas as pd
import requests
import json
import scipy
import requests
import json
import numpy as np

/Users/nuomihan/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
api_key = ''

In [6]:
gmaps = googlemaps.Client(key=api_key)

In [7]:
#import data
DE = pd.read_excel('data/zipcode/DE.xlsx', skiprows=4)
MD = pd.read_excel('data/zipcode/MD.xlsx',skiprows=4)
VA = pd.read_excel('data/zipcode/VA.xlsx',skiprows=4)
WV = pd.read_excel('data/zipcode/WV.xlsx',skiprows=4)
PA = pd.read_excel('data/zipcode/PA.xlsx',skiprows=4)
DC = pd.read_csv('data/zipcode/DC.csv')

In [88]:
#concat all the dataframe into a comprehensive one
allState = pd.concat([DE,MD,VA,WV,PA])
#filter out summary rows for each state
allState = allState[allState['name'].str.isnumeric()]
#select useful cols
allState = allState[['name','state','population']]
allState.rename(columns={'name':'zip'},inplace=True)
allState

,zip,state,population
1,19701,Delaware,44553
2,19702,Delaware,56571
3,19703,Delaware,16376
4,19706,Delaware,1984
5,19707,Delaware,17011
...,...,...,...
1829,19607,Pennsylvania,23975
1830,19608,Pennsylvania,24092
1831,19609,Pennsylvania,10225
1832,19610,Pennsylvania,16401


In [89]:
#add DC to the df
DC=DC[['zip','county','population']]
DC.rename(columns={'county':'state'},inplace=True)
allState = pd.concat([allState, DC])
#reset the index
allState.reset_index(drop=True, inplace=True)
#rank by population
allState.sort_values(by=['population'],ascending=False,inplace=True)

In [90]:
#check the distribution of population
allState.describe()

,population
count,4047.000000
mean,7720.549790
std,12398.034742
min,0.000000
25%,501.000000
50%,2016.000000
75%,8891.500000
max,83796.000000


In [10]:
for i in [DC,DE,MD,VA,WV,PA]:
    print(i['state'].unique())
    print (i['population'].describe())

['District Of Columbia']
count       28.000000
mean     24394.178571
std      21954.261240
min        404.000000
25%       3830.750000
50%      18805.000000
75%      36080.250000
max      68201.000000
Name: population, dtype: float64
['Delaware']
count        69.000000
mean      28807.521739
std      118839.688133
min           0.000000
25%        1984.000000
50%       10273.000000
75%       21437.000000
max      993635.000000
Name: population, dtype: float64
['Maryland']
count    4.780000e+02
mean     2.578026e+04
std      2.816971e+05
min      0.000000e+00
25%      8.662500e+02
50%      4.846500e+03
75%      2.122150e+04
max      6.161707e+06
Name: population, dtype: float64
['Virginia']
count    9.040000e+02
mean     1.908117e+04
std      2.868774e+05
min      0.000000e+00
25%      8.342500e+02
50%      2.718500e+03
75%      1.189400e+04
max      8.624511e+06
Name: population, dtype: float64
['West Virginia']
count    7.390000e+02
mean     4.851930e+03
std      6.604944e+04
min     

In [91]:
#count the zipcode area with population more than 800
sum(allState['population']>=800)

2699

In [122]:
#filter out rows with population less than 800
allState800 = allState[allState['population']>=800]
#reset the index
allState800.reset_index(drop=True, inplace=True)

## Attempt2: using Places(New) 

In [141]:
def restaurantScarping(text):

    # Your API endpoint
    url = 'https://places.googleapis.com/v1/places:searchText'

    # Data to be sent in the POST request
    data = {
      "textQuery": text
    }

    # Headers to be sent with the request
    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': api_key,  # Replace 'api_key' with your actual API key
        'X-Goog-FieldMask': 'places.displayName,places.id,places.formattedAddress,places.nationalPhoneNumber,places.userRatingCount,places.rating'
    }

    # Making the POST request
    response = requests.post(url, json=data, headers=headers)
    
        # Check if the response contains no value
    if not response.json().get('places'):
        return None

    return response.json()['places']
    

In [85]:
def decodeResult(json):
    for place in json:
        new_row = {'name':place['displayName']['text'], 'id':place['id'], 
                   'address':place['formattedAddress'], 'phoneNumber':place['nationalPhoneNumber'], 
                   'rating':place['rating'],'ratesCount': place['userRatingCount']}
        restaurants = pd.concat([restaurants, pd.DataFrame(data=new_row,index=['0'])], ignore_index=True)

In [83]:
#create a df to store results
restaurants = pd.DataFrame(columns=['name','id','address','phoneNumber','rating','ratesCount'])

In [142]:
for i in range(len(allState800)):
    queryText = 'chinese restaurant or sushi restaurants in'+str(allState800['zip'][i])
    places = restaurantScarping(queryText)
    # Skip to the next iteration if no places are found
    if places is None:
        continue
    for place in places:
        new_row = {
        'name': place.get('displayName', {}).get('text', None),  # Nested dictionary access
        'id': place.get('id', None),
        'address': place.get('formattedAddress', None),
        'phoneNumber': place.get('nationalPhoneNumber', None),
        'rating': place.get('rating', None),
        'ratesCount': place.get('userRatingCount', None)}
        restaurants = pd.concat([restaurants, pd.DataFrame(data=new_row,index=['0'])], ignore_index=True)
    

In [146]:
#drop the duplicated rows
uniqueRes = restaurants.drop_duplicates(subset=['id'])

In [147]:
uniqueRes.to_excel('data/result1.xlsx')

In [90]:
#url='https://www.google.com/maps/search/?api=1&query=chinese+restaurant+19702'
#response = requests.get(url)
